In [108]:
import os
import pandas as pd
import numpy as np
import gensim.downloader
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize 

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor

In [45]:
word_vector = gensim.downloader.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [97]:
data_folder = "data/preprocessed"

### load train df

In [156]:
df = pd.read_csv(os.path.join(data_folder, "lcp_single_train_preprocessed.csv"), index_col=0)
df.head()

,corpus,sentence,token,complexity
id,,,,
3ZLW647WALVGE8EBR50EGUBPU4P32A,bible,bible behold came river seven cattle sleek fat...,river,0.000000
34R0BODSP1ZBN3DVY8J8XSIY551E5C,bible,bible fellow bondservant brother prophet keep ...,brother,0.000000
3S1WOPCJFGTJU2SGNAN2Y213N6WJE3,bible,bible man lord land said u know honest men lea...,brother,0.050000
3BFNCI9LYKQN09BHXHH9CLSX5KP738,bible,bible shimei sixteen son six daughter brother ...,brother,0.150000
3G5RUKN2EC3YIWSKUXZ8ZVH95R49N2,bible,bible put brother far,brother,0.263889


In [157]:
# take mean of duplicate tokens
# token not in vocab are considered as 'unk'

df = df.groupby('token').mean().reset_index()
print("unique tokens : {}".format(len(df)))

count = [True if w in word_vector else False for w in df['token']]
print("{} tokens not in vocab".format(len(tokens)-sum(count)))

vectors = np.array(list(df['token'].apply(lambda x:word_vector[x] if x in word_vector else word_vector['unk'])))

unique tokens : 2856
11 tokens not in vocab


### load test df

In [159]:
test_df = pd.read_csv(os.path.join(data_folder, "lcp_single_test_preprocessed.csv"), index_col=0)
count = [True if w in word_vector else False for w in test_df['token']]
print("{} tokens not in vocab".format(len(test_df)-sum(count)))

testdf_vectors = np.array(list(test_df['token'].apply(lambda x:word_vector[x] if x in word_vector else word_vector['unk'])))

35 tokens not in vocab


In [148]:
reg = GradientBoostingRegressor().fit(vectors, np.array(df['complexity']))

In [161]:
y_pred = reg.predict(testdf_vectors)

In [168]:
pred = pd.DataFrame({"ID":test_df.index, "complexity":y_pred})
pred.to_csv("GB.csv", index=False, header=False)